<a href="https://www.kaggle.com/code/averma111/image-matching-with-keras?scriptVersionId=126333429" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!wget http://bit.ly/3ZLyF82 -O CSS.css -q
    
from IPython.core.display import HTML
with open('./CSS.css', 'r') as file:
    custom_css = file.read()

HTML(custom_css)

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.layers import Input
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

In [3]:
def image_data_preprocessing(data_path,label_path):
    image_list=[]
    pose_list=[]
    labels = pd.read_csv(label_path)
    for _, row in labels.iterrows():
        img_path = os.path.join(data_path, row['image_path'])
        img = cv2.imread(img_path)
        if img is None:
            print(f"Failed to load image: {img_path}")
            continue
        img = cv2.resize(img, (224, 224)) 
        img = img / 255.0  
        rotation = np.array(row['rotation_matrix'].split(';'), dtype=float).reshape(3, 3)
        translation = np.array(row['translation_vector'].split(';'), dtype=float)
        pose = np.hstack((rotation, translation.reshape(-1, 1)))
        pose = pose.flatten() 
        image_list.append(img)
        pose_list.append(pose)
    images = np.array(image_list)
    poses = np.array(pose_list)

    return images, poses

In [4]:
ROOT_PATH='/kaggle/input/image-matching-challenge-2023'
data_dir = ROOT_PATH+'/train'
labels_path = ROOT_PATH+'/train/train_labels.csv'
images, poses = image_data_preprocessing(data_dir, labels_path)


In [5]:
X_train, X_val, y_train, y_val = train_test_split(images, poses, test_size=0.2, random_state=42)
X_train.shape
X_val.shape
y_train.shape
y_val.shape

(66, 12)

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Flatten

data_gen = ImageDataGenerator(rotation_range=20,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              shear_range=0.2,
                              zoom_range=0.2,
                              horizontal_flip=True,
                              fill_mode='nearest')

data_gen.fit(X_train)

In [7]:
def create_model():
    input_img = Input(shape=(224, 224, 3))
    evaluate = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
    evaluate = Conv2D(64, (3, 3), activation='relu', padding='same')(evaluate)
    evaluate = MaxPooling2D((2, 2), padding='same')(evaluate)
    evaluate = Conv2D(128, (3, 3), activation='relu', padding='same')(evaluate)
    evaluate = Conv2D(128, (3, 3), activation='relu', padding='same')(evaluate)
    evaluate = MaxPooling2D((2, 2), padding='same')(evaluate)
    evaluate = Conv2D(256, (3, 3), activation='relu', padding='same')(evaluate)
    evaluate = Conv2D(256, (3, 3), activation='relu', padding='same')(evaluate)
    evaluate = MaxPooling2D((2, 2), padding='same')(evaluate)
    evaluate = Conv2D(512, (3, 3), activation='relu', padding='same')(evaluate)
    evaluate = Conv2D(512, (3, 3), activation='relu', padding='same')(evaluate)
    evaluate = MaxPooling2D((2, 2), padding='same')(evaluate)
    evaluate = Flatten()(evaluate)
    evaluate = Dense(1024, activation='relu')(evaluate)
    evaluate = Dense(512, activation='relu')(evaluate)
    pose_output = Dense(12, activation=None)(evaluate)

    model = Model(inputs=input_img, outputs=pose_output)
    model.compile(optimizer='adam', loss='mean_squared_error')

    return model

In [8]:
initial_learning_rate = 1e-5

model = create_model()
model.compile(optimizer=Adam(learning_rate=initial_learning_rate), loss='mean_squared_error',metrics=['mae'])


In [9]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     147584    
                                                             

In [13]:
history = model.fit(data_gen.flow(X_train, y_train, batch_size=1),
                    validation_data=(X_val, y_val),
                    epochs=100,
                    steps_per_epoch=len(X_train)//1)

Epoch 1/100
261/261 [==============================] - 10s 39ms/step - loss: 18.6745 - mae: 1.7021 - val_loss: 18.5207 - val_mae: 1.8513
Epoch 2/100
261/261 [==============================] - 10s 40ms/step - loss: 17.8311 - mae: 1.6521 - val_loss: 17.4109 - val_mae: 1.7524
Epoch 3/100
261/261 [==============================] - 10s 39ms/step - loss: 17.8909 - mae: 1.6614 - val_loss: 17.9619 - val_mae: 1.7091
Epoch 4/100
261/261 [==============================] - 10s 39ms/step - loss: 17.6834 - mae: 1.6327 - val_loss: 18.4330 - val_mae: 1.7151
Epoch 5/100
261/261 [==============================] - 10s 39ms/step - loss: 17.2775 - mae: 1.6334 - val_loss: 17.7330 - val_mae: 1.8000
Epoch 6/100
261/261 [==============================] - 10s 39ms/step - loss: 17.7068 - mae: 1.6509 - val_loss: 19.0678 - val_mae: 1.8853
Epoch 7/100
261/261 [==============================] - 10s 39ms/step - loss: 17.7441 - mae: 1.6413 - val_loss: 24.5852 - val_mae: 2.1523
Epoch 8/100
261/261 [====================

KeyboardInterrupt: 

In [11]:
def preprocess_test_data(data_dir, image_paths):
    images = []
    for image_path in image_paths:
        img_path = os.path.join(data_dir, image_path)
        img = cv2.imread(img_path)
        if img is None:
            print(f"Failed to load image: {img_path}")
            continue
        img = cv2.resize(img, (224, 224))  
        img = img / 255.0  
        images.append(img)
    images = np.array(images)
    return images

In [12]:
test_image_paths = sample_submission_df['image_path'].values
test_data_dir = "/kaggle/input/image-matching-challenge-2023/test"
test_images_processed = preprocess_test_data(test_data_dir, test_image_paths)
predicted_poses = model.predict(test_images_processed)
sample_submission_df.iloc[:, 1:] = predicted_poses
sample_submission_df.to_csv("submission.csv", index=False)

Failed to load image: /kaggle/input/image-matching-challenge-2023/test/2cfa01ab573141e4/2fa124afd1f74f38/images/093870b65ebf4ded.png
Failed to load image: /kaggle/input/image-matching-challenge-2023/test/2cfa01ab573141e4/2fa124afd1f74f38/images/11c073bfeee04544.png
Failed to load image: /kaggle/input/image-matching-challenge-2023/test/2cfa01ab573141e4/2fa124afd1f74f38/images/11d5430cd9e246d4.png


ValueError: Unexpected result of `predict_function` (Empty batch_outputs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.